In [1]:
import os
from glob import glob

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping


%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
X_train = np.load("256_192_train.npy")
y_train = np.load("train_labels.npy")
X_val = np.load("256_192_val.npy")
y_val = np.load("val_labels.npy")

X_train.shape, X_val.shape
y_train.shape, y_val.shape

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_train.shape, y_val.shape


In [3]:
pre_trained_model = VGG16(input_shape=(192, 256, 3), include_top=False, weights="imagenet")
for layer in pre_trained_model.layers:
    print(layer.name)
    layer.trainable = False
    
print(len(pre_trained_model.layers))

58889256/58889256 [==============================] - 52s 1us/step
input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool
19


In [4]:
last_layer = pre_trained_model.get_layer('block5_pool')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 6, 8, 512)


In [5]:
# Flatten the output layer to 1 dimension
x = layers.GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(7, activation='softmax')(x)

# Configure and compile the model

model = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 192, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 192, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 192, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 96, 128, 64)       0         
                                                                 
 block2_conv1 (Conv2D)       (None, 96, 128, 128)      73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 96, 128, 128)      147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 48, 64, 128)       0     

C:\Users\chama\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [6]:
train_datagen = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)
batch_size = 64
epochs = 3
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size), 
                              validation_steps=(X_val.shape[0] // batch_size))

C:\Users\chama\AppData\Local\Temp\ipykernel_8308\3698051878.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),


Epoch 1/3
126/126 [==============================] - 2038s 16s/step - loss: 1.4026 - accuracy: 0.5834 - val_loss: 1.0571 - val_accuracy: 0.6696
Epoch 2/3
126/126 [==============================] - 1994s 16s/step - loss: 1.1078 - accuracy: 0.6612 - val_loss: 1.0040 - val_accuracy: 0.6696
Epoch 3/3
126/126 [==============================] - 2009s 16s/step - loss: 1.0385 - accuracy: 0.6676 - val_loss: 1.0132 - val_accuracy: 0.6540


In [7]:
for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers[15:]:
    layer.trainable = True
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 192, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 192, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 192, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 96, 128, 64)       0         
                                                                 
 block2_conv1 (Conv2D)       (None, 96, 128, 128)      73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 96, 128, 128)      147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 48, 64, 128)       0     

In [8]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=3)
batch_size = 64
epochs = 30
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size), callbacks=[learning_rate_reduction])

C:\Users\chama\AppData\Local\Temp\ipykernel_8308\1422572830.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),


Epoch 1/30
126/126 [==============================] - 2442s 19s/step - loss: 0.9208 - acc: 0.6887 - val_loss: 0.8281 - val_acc: 0.6830 - lr: 1.0000e-04
Epoch 2/30
126/126 [==============================] - 2467s 20s/step - loss: 0.7668 - acc: 0.7228 - val_loss: 0.8067 - val_acc: 0.6897 - lr: 1.0000e-04
Epoch 3/30
126/126 [==============================] - 2472s 20s/step - loss: 0.7190 - acc: 0.7421 - val_loss: 0.7671 - val_acc: 0.7076 - lr: 1.0000e-04
Epoch 4/30
126/126 [==============================] - 2555s 20s/step - loss: 0.6819 - acc: 0.7512 - val_loss: 0.7166 - val_acc: 0.7321 - lr: 1.0000e-04
Epoch 5/30
126/126 [==============================] - 2622s 21s/step - loss: 0.6446 - acc: 0.7661 - val_loss: 0.7310 - val_acc: 0.7076 - lr: 1.0000e-04
Epoch 6/30
126/126 [==============================] - 2476s 20s/step - loss: 0.6250 - acc: 0.7706 - val_loss: 0.7496 - val_acc: 0.7188 - lr: 1.0000e-04
Epoch 7/30
126/126 [==============================] - 2861s 23s/step - loss: 0.5984 - ac

In [9]:
loss_val, acc_val = model.evaluate(X_val, y_val, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))
X_test = np.load("256_192_test.npy")
y_test = np.load("test_labels.npy")
y_test = to_categorical(y_test)
loss_test, acc_test = model.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))

29/29 [==============================] - 207s 7s/step - loss: 0.6209 - acc: 0.7905
Validation: accuracy = 0.790466  ;  loss_v = 0.620855
32/32 [==============================] - 229s 7s/step - loss: 0.6150 - acc: 0.8034
Test: accuracy = 0.803393  ;  loss = 0.615045


In [10]:
model.save("VGG16.h5")